In [6]:
"""
═══════════════════════════════════════════════════════════════════════
RDD2022 BASELINE TRAINING - YOLOv8n @ 640
═══════════════════════════════════════════════════════════════════════
Model: YOLOv8n (Nano - fastest, baseline)
Resolution: 640×640
Epochs: 100
Dataset: 38,385 images (RDD2022)
Target: 45-50% mAP50
═══════════════════════════════════════════════════════════════════════
"""

import os
import sys

print("="*70)
print("RDD2022 - BASELINE TRAINING (YOLOv8n @ 640)")
print("="*70)

# ✅ CRITICAL: Fix NumPy FIRST before importing anything
print("\n📦 Step 1: Fixing NumPy compatibility...")
print("   Uninstalling NumPy 2.x...")
!pip uninstall -y -q numpy

print("   Installing NumPy 1.26.4 + Ultralytics...")
!pip install -q "numpy==1.26.4" ultralytics

print("\n✅ Step 2: Verifying installation...")

# Now import everything
from pathlib import Path
import shutil
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import cv2
import torch
import pandas as pd

print(f"\n📊 Package Versions:")
print(f"   - NumPy:   {np.__version__} {'✅' if np.__version__.startswith('1') else '❌'}")
print(f"   - OpenCV:  {cv2.__version__}")
print(f"   - PyTorch: {torch.__version__}")
print(f"   - CUDA:    {torch.cuda.is_available()}")

# ✅ Test NumPy/OpenCV compatibility
try:
    test_img = np.zeros((10, 10, 3), dtype=np.uint8)
    resized = cv2.resize(test_img, (20, 20))
    print(f"   - OpenCV/NumPy: ✅ PASSED")
except Exception as e:
    print(f"   - OpenCV/NumPy: ❌ FAILED ({e})")
    print("   ⚠️  RESTART KERNEL and run this cell again!")

# ✅ Disable WandB
os.environ['WANDB_DISABLED'] = 'true'
os.environ['WANDB_MODE'] = 'disabled'
os.environ['WANDB_SILENT'] = 'true'

from ultralytics import YOLO, settings
settings.update({'wandb': False})

if torch.cuda.is_available():
    print(f"\n🎮 GPU Information:")
    print(f"   - Device: {torch.cuda.get_device_name(0)}")
    print(f"   - Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

print("\n" + "="*70)
print("✅ SETUP COMPLETE - Ready for Training!")
print("="*70 + "\n")

RDD2022 - BASELINE TRAINING (YOLOv8n @ 640)

📦 Step 1: Fixing NumPy compatibility...
   Uninstalling NumPy 2.x...
   Installing NumPy 1.26.4 + Ultralytics...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.4.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.7, but you have notebook 6.5.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.5 which is incompatible.
google-colab 1.0.0 requires tornado==6.4.2, but you have tornado 6.5.2 which is incompatible.
dopamine-r

In [7]:
"""
DATASET CONFIGURATION
Verify paths and create data.yaml
"""

# Dataset paths
DATASET_BASE = Path("/kaggle/input/rdd-2022/RDD_SPLIT")

# Verify dataset exists
print("📂 Verifying Dataset...\n")

if not DATASET_BASE.exists():
    print("❌ Dataset not found!")
    print("💡 Make sure to add 'rdd-2022' dataset to this notebook")
    print("\nSearching for dataset...")
    for item in Path("/kaggle/input/").iterdir():
        print(f"   Found: {item.name}")
else:
    print(f"✅ Dataset found: {DATASET_BASE}\n")

splits = ['train', 'val', 'test']
dataset_info = {}

for split in splits:
    img_dir = DATASET_BASE / split / "images"
    lbl_dir = DATASET_BASE / split / "labels"
    
    n_images = len(list(img_dir.glob("*.jpg"))) if img_dir.exists() else 0
    n_labels = len(list(lbl_dir.glob("*.txt"))) if lbl_dir.exists() else 0
    
    dataset_info[split] = {'images': n_images, 'labels': n_labels}
    
    status = "✅" if n_images > 0 and n_labels > 0 else "❌"
    print(f"{status} {split:8s}: {n_images:6,} images, {n_labels:6,} labels")

# Class names
CLASS_NAMES = [
    'longitudinal crack',
    'transverse crack', 
    'alligator crack',
    'other corruption',
    'Pothole'
]

print(f"\n🏷️  Damage Classes ({len(CLASS_NAMES)}):")
for i, name in enumerate(CLASS_NAMES):
    print(f"   {i}: {name}")

# Create data.yaml in working directory
data_yaml_content = f"""# RDD2022 Dataset Configuration
# Baseline Training: YOLOv8n @ 640
# Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

path: {DATASET_BASE}
train: train/images
val: val/images
test: test/images

nc: 5
names:
  0: longitudinal crack
  1: transverse crack
  2: alligator crack
  3: other corruption
  4: Pothole
"""

data_yaml_path = Path("/kaggle/working/data.yaml")
with open(data_yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"\n✅ data.yaml created: {data_yaml_path}")
print("\n" + "="*70 + "\n")

📂 Verifying Dataset...

✅ Dataset found: /kaggle/input/rdd-2022/RDD_SPLIT

✅ train   : 26,869 images, 26,869 labels
✅ val     :  5,758 images,  5,758 labels
✅ test    :  5,758 images,  5,758 labels

🏷️  Damage Classes (5):
   0: longitudinal crack
   1: transverse crack
   2: alligator crack
   3: other corruption
   4: Pothole

✅ data.yaml created: /kaggle/working/data.yaml




In [8]:
"""
RESULTS SAVER - NEVER LOSE TRAINING RESULTS!
Automatically saves everything after training
"""

class TrainingResultsSaver:
    """
    Bulletproof results saver for Kaggle training
    Saves weights, metrics, plots, and creates downloadable archive
    """
    
    def __init__(self, experiment_name):
        self.exp_name = experiment_name
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Create save directory
        self.save_dir = Path(f"/kaggle/working/{experiment_name}_{self.timestamp}")
        self.save_dir.mkdir(parents=True, exist_ok=True)
        
        self.metadata = {
            'experiment': experiment_name,
            'timestamp': self.timestamp,
            'status': 'initialized'
        }
        
        print(f"💾 ResultsSaver initialized: {self.save_dir}")
    
    def save_training_run(self, model, results_dir, config):
        """
        Save complete training run
        
        Args:
            model: Trained YOLO model
            results_dir: Path to training results (e.g., 'runs/detect/train')
            config: Training configuration dict
        """
        print(f"\n{'='*70}")
        print(f"💾 SAVING TRAINING RESULTS: {self.exp_name}")
        print(f"{'='*70}\n")
        
        results_path = Path(results_dir)
        
        # 1. Save weights
        weights_dir = results_path / "weights"
        if weights_dir.exists():
            save_weights_dir = self.save_dir / "weights"
            save_weights_dir.mkdir(exist_ok=True)
            
            for weight_file in ["best.pt", "last.pt"]:
                src = weights_dir / weight_file
                if src.exists():
                    shutil.copy(src, save_weights_dir / weight_file)
                    file_size_mb = src.stat().st_size / (1024 * 1024)
                    print(f"✅ Saved: weights/{weight_file} ({file_size_mb:.2f} MB)")
        
        # 2. Save results.csv
        results_csv = results_path / "results.csv"
        if results_csv.exists():
            shutil.copy(results_csv, self.save_dir / "results.csv")
            print(f"✅ Saved: results.csv")
            
            # Parse final metrics
            df = pd.read_csv(results_csv)
            final_metrics = df.iloc[-1].to_dict()
            
            # Handle different column naming conventions
            map50_col = 'metrics/mAP50(B)' if 'metrics/mAP50(B)' in final_metrics else 'metrics/mAP_0.5'
            map_col = 'metrics/mAP50-95(B)' if 'metrics/mAP50-95(B)' in final_metrics else 'metrics/mAP_0.5:0.95'
            precision_col = 'metrics/precision(B)' if 'metrics/precision(B)' in final_metrics else 'metrics/precision'
            recall_col = 'metrics/recall(B)' if 'metrics/recall(B)' in final_metrics else 'metrics/recall'
            
            self.metadata['final_metrics'] = {
                'mAP50': float(final_metrics.get(map50_col, 0)),
                'mAP50-95': float(final_metrics.get(map_col, 0)),
                'precision': float(final_metrics.get(precision_col, 0)),
                'recall': float(final_metrics.get(recall_col, 0)),
            }
            
            print(f"\n📊 Final Metrics:")
            print(f"   mAP@50:    {self.metadata['final_metrics']['mAP50']:.4f}")
            print(f"   mAP@50-95: {self.metadata['final_metrics']['mAP50-95']:.4f}")
            print(f"   Precision: {self.metadata['final_metrics']['precision']:.4f}")
            print(f"   Recall:    {self.metadata['final_metrics']['recall']:.4f}")
        
        # 3. Save plots
        plots_dir = self.save_dir / "plots"
        plots_dir.mkdir(exist_ok=True)
        
        plot_files = [
            "results.png", "confusion_matrix.png", "confusion_matrix_normalized.png",
            "F1_curve.png", "P_curve.png", "R_curve.png", "PR_curve.png",
            "labels.jpg", "labels_correlogram.jpg", "train_batch0.jpg",
            "train_batch1.jpg", "train_batch2.jpg", "val_batch0_labels.jpg",
            "val_batch0_pred.jpg", "val_batch1_labels.jpg", "val_batch1_pred.jpg",
            "val_batch2_labels.jpg", "val_batch2_pred.jpg"
        ]
        
        saved_plots = 0
        for plot in plot_files:
            src = results_path / plot
            if src.exists():
                shutil.copy(src, plots_dir / plot)
                saved_plots += 1
        
        print(f"\n✅ Saved: plots/ ({saved_plots} files)")
        
        # 4. Save configuration
        config_path = self.save_dir / "config.json"
        with open(config_path, 'w') as f:
            # Convert Path objects to strings for JSON serialization
            config_serializable = {k: str(v) if isinstance(v, Path) else v for k, v in config.items()}
            json.dump(config_serializable, f, indent=4)
        print(f"✅ Saved: config.json")
        
        # 5. Save metadata
        self.metadata['status'] = 'completed'
        self.metadata['config'] = config_serializable
        
        metadata_path = self.save_dir / "metadata.json"
        with open(metadata_path, 'w') as f:
            json.dump(self.metadata, f, indent=4)
        print(f"✅ Saved: metadata.json")
        
        # 6. Create results summary
        self._create_summary()
        
        print(f"\n{'='*70}")
        print(f"✅ ALL RESULTS SAVED TO: {self.save_dir}")
        print(f"{'='*70}\n")
        
        return self.save_dir
    
    def _create_summary(self):
        """Create human-readable summary"""
        summary = f"""
{'='*70}
TRAINING SUMMARY: {self.exp_name}
{'='*70}
Timestamp: {self.timestamp}
Status: {self.metadata['status']}

FINAL METRICS:
"""
        if 'final_metrics' in self.metadata:
            metrics = self.metadata['final_metrics']
            summary += f"  mAP@50:      {metrics['mAP50']:.4f} ({metrics['mAP50']*100:.2f}%)\n"
            summary += f"  mAP@50-95:   {metrics['mAP50-95']:.4f} ({metrics['mAP50-95']*100:.2f}%)\n"
            summary += f"  Precision:   {metrics['precision']:.4f} ({metrics['precision']*100:.2f}%)\n"
            summary += f"  Recall:      {metrics['recall']:.4f} ({metrics['recall']*100:.2f}%)\n"
        
        summary += f"\nCONFIGURATION:\n"
        if 'config' in self.metadata:
            key_params = ['model', 'epochs', 'imgsz', 'batch', 'patience', 'optimizer', 'amp']
            for key in key_params:
                if key in self.metadata['config']:
                    summary += f"  {key:15s}: {self.metadata['config'][key]}\n"
        
        summary += f"\n{'='*70}\n"
        
        summary_path = self.save_dir / "SUMMARY.txt"
        with open(summary_path, 'w') as f:
            f.write(summary)
        
        print(summary)
        
        return summary_path
    
    def create_archive(self):
        """Create ZIP archive for download"""
        import zipfile
        
        zip_path = f"{self.save_dir}.zip"
        
        print(f"\n📦 Creating downloadable archive...")
        
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for file in self.save_dir.rglob('*'):
                if file.is_file():
                    arcname = file.relative_to(self.save_dir.parent)
                    zipf.write(file, arcname)
        
        size_mb = Path(zip_path).stat().st_size / (1024 * 1024)
        print(f"✅ Archive created: {Path(zip_path).name}")
        print(f"📊 Size: {size_mb:.2f} MB")
        print(f"\n{'='*70}")
        print(f"🚨 DOWNLOAD THIS FILE IMMEDIATELY!")
        print(f"{'='*70}")
        print(f"1. Look in /kaggle/working/ folder (left sidebar)")
        print(f"2. Find: {Path(zip_path).name}")
        print(f"3. Right-click → Download")
        print(f"4. Save to your local machine NOW!")
        print(f"{'='*70}\n")
        
        return zip_path


print("✅ TrainingResultsSaver class defined")

✅ TrainingResultsSaver class defined


In [9]:
"""
TRAINING CONFIGURATION
Optimized with AMP and Plots enabled
"""

# Experiment name
EXPERIMENT_NAME = "baseline_yolov8n_640"

# Training configuration - FULLY OPTIMIZED
TRAINING_CONFIG = {
    'model': 'yolov8n.pt',
    'data': str(data_yaml_path),
    'epochs': 100,
    'imgsz': 640,
    'batch': 16,
    'device': 0,
    'project': '/kaggle/working/runs',
    'name': 'train',
    'patience': 20,
    'save': True,
    'save_period': 10,
    'cache': True,
    'workers': 8,
    'optimizer': 'auto',
    'verbose': True,
    'seed': 42,
    'deterministic': True,
    'single_cls': False,
    'rect': False,
    'cos_lr': False,
    'close_mosaic': 10,
    'resume': False,
    'amp': True,        # ✅ ENABLED - Faster training (15% speedup)
    'plots': True,      # ✅ ENABLED - Auto-generate training plots
    'fraction': 1.0,
    'profile': False,
    'freeze': None,
    'lr0': 0.01,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'box': 7.5,
    'cls': 0.5,
    'dfl': 1.5,
    'val': True,
}

print("="*70)
print("TRAINING CONFIGURATION - OPTIMIZED")
print("="*70)
print(f"\n🔧 Model Configuration:")
print(f"   Model:       {TRAINING_CONFIG['model']}")
print(f"   Resolution:  {TRAINING_CONFIG['imgsz']}×{TRAINING_CONFIG['imgsz']}")
print(f"   Epochs:      {TRAINING_CONFIG['epochs']}")
print(f"   Batch Size:  {TRAINING_CONFIG['batch']}")
print(f"   Device:      GPU {TRAINING_CONFIG['device']}")
print(f"   Patience:    {TRAINING_CONFIG['patience']} epochs")
print(f"   AMP:         {TRAINING_CONFIG['amp']} ✅ (Fast training)")
print(f"   Plots:       {TRAINING_CONFIG['plots']} ✅ (Auto-generated)")

print(f"\n📊 Dataset:")
print(f"   Train:       {dataset_info['train']['images']:,} images")
print(f"   Val:         {dataset_info['val']['images']:,} images")
print(f"   Test:        {dataset_info['test']['images']:,} images")
print(f"   Classes:     {len(CLASS_NAMES)}")

print(f"\n⚡ Performance:")
print(f"   Estimated Time: ~6-8 hours (with AMP)")
print(f"   Speed Boost:    ~15% faster than without AMP")
print(f"   Plots:          15+ auto-generated visualizations")

print(f"\n🎯 Target: 45-50% mAP@50")
print("\n" + "="*70 + "\n")

TRAINING CONFIGURATION - OPTIMIZED

🔧 Model Configuration:
   Model:       yolov8n.pt
   Resolution:  640×640
   Epochs:      100
   Batch Size:  16
   Device:      GPU 0
   Patience:    20 epochs
   AMP:         True ✅ (Fast training)
   Plots:       True ✅ (Auto-generated)

📊 Dataset:
   Train:       26,869 images
   Val:         5,758 images
   Test:        5,758 images
   Classes:     5

⚡ Performance:
   Estimated Time: ~6-8 hours (with AMP)
   Speed Boost:    ~15% faster than without AMP
   Plots:          15+ auto-generated visualizations

🎯 Target: 45-50% mAP@50




In [ ]:
"""
LOAD MODEL AND START TRAINING
⚠️ This cell will run for approximately 6-8 hours
⚠️ Do NOT close this notebook while training!
"""

print("="*70)
print("STARTING BASELINE TRAINING")
print("="*70)

# Initialize results saver
saver = TrainingResultsSaver(EXPERIMENT_NAME)

# Load pretrained YOLOv8n model
print("\n📥 Loading YOLOv8n pretrained model...")
model = YOLO(TRAINING_CONFIG['model'])

print(f"✅ Model loaded: {TRAINING_CONFIG['model']}")

# Count model parameters
total_params = sum(p.numel() for p in model.model.parameters())
trainable_params = sum(p.numel() for p in model.model.parameters() if p.requires_grad)

print(f"📊 Model Information:")
print(f"   Total Parameters:     {total_params:,}")
print(f"   Trainable Parameters: {trainable_params:,}")
print(f"   Model Size:           ~6.3 MB")

# Start training
print(f"\n{'='*70}")
print(f"🚀 STARTING TRAINING")
print(f"{'='*70}")
print(f"\n⏱️  Estimated Duration: 6-8 hours")
print(f"📊 Progress will be shown below")
print(f"⚠️  Do NOT close this notebook!")
print(f"\n{'='*70}\n")

# Record start time
start_time = datetime.now()
print(f"⏰ Training started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}\n")

# Train the model
try:
    results = model.train(**TRAINING_CONFIG)
    
    # Record end time
    end_time = datetime.now()
    duration = end_time - start_time
    
    print(f"\n{'='*70}")
    print(f"🎉 TRAINING COMPLETE!")
    print(f"{'='*70}")
    print(f"⏰ Started:  {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"⏰ Finished: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"⏱️  Duration: {duration}")
    print(f"{'='*70}\n")
    
    # Get results directory
    results_dir = Path(TRAINING_CONFIG['project']) / TRAINING_CONFIG['name']
    print(f"📁 Results directory: {results_dir}")
    
    # Verify results exist
    if results_dir.exists():
        print(f"✅ Training results saved successfully")
    else:
        print(f"⚠️  Warning: Results directory not found")
        
except Exception as e:
    print(f"\n{'='*70}")
    print(f"❌ ERROR DURING TRAINING")
    print(f"{'='*70}")
    print(f"Error: {str(e)}")
    print(f"\nPlease check:")
    print(f"  1. GPU is enabled")
    print(f"  2. Dataset paths are correct")
    print(f"  3. Enough GPU memory (try reducing batch size)")
    raise

STARTING BASELINE TRAINING
💾 ResultsSaver initialized: /kaggle/working/baseline_yolov8n_640_20251209_195636

📥 Loading YOLOv8n pretrained model...
✅ Model loaded: yolov8n.pt
📊 Model Information:
   Total Parameters:     3,157,200
   Trainable Parameters: 0
   Model Size:           ~6.3 MB

🚀 STARTING TRAINING

⏱️  Estimated Duration: 6-8 hours
📊 Progress will be shown below
⚠️  Do NOT close this notebook!


⏰ Training started at: 2025-12-09 19:56:36

Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=

In [ ]:
"""
SAVE ALL RESULTS
🚨 RUN THIS CELL IMMEDIATELY AFTER TRAINING COMPLETES!
"""

print("="*70)
print("SAVING TRAINING RESULTS")
print("="*70 + "\n")

# Verify results directory exists
results_dir = Path(TRAINING_CONFIG['project']) / TRAINING_CONFIG['name']

if not results_dir.exists():
    print(f"❌ Results directory not found: {results_dir}")
    print(f"\nSearching for results...")
    
    runs_dir = Path(TRAINING_CONFIG['project'])
    if runs_dir.exists():
        for subdir in runs_dir.iterdir():
            if subdir.is_dir():
                print(f"   Found: {subdir}")
else:
    print(f"✅ Results directory found: {results_dir}\n")
    
    # Save all training results
    save_dir = saver.save_training_run(
        model=model,
        results_dir=results_dir,
        config=TRAINING_CONFIG
    )
    
    # Create downloadable archive
    zip_path = saver.create_archive()
    
    print("✅ Results saved successfully!")
    print(f"\n📥 DOWNLOAD INSTRUCTIONS:")
    print(f"   1. Look at the left sidebar → Files")
    print(f"   2. Navigate to /kaggle/working/")
    print(f"   3. Find file: {Path(zip_path).name}")
    print(f"   4. Right-click → Download")
    print(f"   5. Save to your computer NOW!")
    print(f"\n🚨 DO THIS BEFORE CLOSING THE NOTEBOOK!")
    print("\n" + "="*70 + "\n")

In [ ]:
"""
VALIDATION ON TEST SET
Evaluate the trained model on held-out test data
"""

print("="*70)
print("VALIDATION ON TEST SET")
print("="*70 + "\n")

# Load best model
best_model_path = save_dir / "weights" / "best.pt"

if not best_model_path.exists():
    print(f"❌ Best model not found: {best_model_path}")
else:
    print(f"✅ Loading best model: {best_model_path}\n")
    model_best = YOLO(str(best_model_path))
    
    # Validate on test set
    print("📊 Running validation on test set...")
    print("   This will take approximately 10-15 minutes\n")
    
    test_results = model_best.val(
        data=str(data_yaml_path),
        split='test',
        imgsz=640,
        batch=16,
        device=0,
        plots=True,
        save_json=True,
        save_hybrid=False,
        conf=0.001,
        iou=0.6,
        max_det=300,
        half=False,
        dnn=False,
        project='/kaggle/working/runs',
        name='val_test',
        verbose=True
    )
    
    print("\n" + "="*70)
    print("TEST SET RESULTS")
    print("="*70)
    print(f"mAP@50:      {test_results.box.map50:.4f} ({test_results.box.map50*100:.2f}%)")
    print(f"mAP@50-95:   {test_results.box.map:.4f} ({test_results.box.map*100:.2f}%)")
    print(f"Precision:   {test_results.box.mp:.4f} ({test_results.box.mp*100:.2f}%)")
    print(f"Recall:      {test_results.box.mr:.4f} ({test_results.box.mr*100:.2f}%)")
    print("="*70 + "\n")
    
    # Save test results
    test_results_dir = Path('/kaggle/working/runs/val_test')
    if test_results_dir.exists():
        test_save_dir = save_dir / "test_results"
        test_save_dir.mkdir(exist_ok=True)
        
        for file in test_results_dir.glob('*'):
            if file.is_file():
                shutil.copy(file, test_save_dir / file.name)
        
        print(f"✅ Test results saved to: {test_save_dir}")
        
        # Update archive with test results
        print("\n📦 Updating archive with test results...")
        zip_path = saver.create_archive()

In [ ]:
"""
ANALYZE TRAINING RESULTS
Generate performance metrics and comparisons
"""

print("="*70)
print("RESULTS ANALYSIS")
print("="*70 + "\n")

# Load training results
results_csv_path = save_dir / "results.csv"

if not results_csv_path.exists():
    print(f"❌ Results CSV not found: {results_csv_path}")
else:
    df_results = pd.read_csv(results_csv_path)
    
    # Get final epoch metrics
    final_epoch = df_results.iloc[-1]
    
    # Determine column names (handle different Ultralytics versions)
    map50_col = 'metrics/mAP50(B)' if 'metrics/mAP50(B)' in df_results.columns else 'metrics/mAP_0.5'
    map_col = 'metrics/mAP50-95(B)' if 'metrics/mAP50-95(B)' in df_results.columns else 'metrics/mAP_0.5:0.95'
    precision_col = 'metrics/precision(B)' if 'metrics/precision(B)' in df_results.columns else 'metrics/precision'
    recall_col = 'metrics/recall(B)' if 'metrics/recall(B)' in df_results.columns else 'metrics/recall'
    
    # Create results table
    results_table = pd.DataFrame({
        'Experiment': [EXPERIMENT_NAME],
        'Model': ['YOLOv8n'],
        'Resolution': [640],
        'Batch': [TRAINING_CONFIG['batch']],
        'Epochs_Trained': [int(final_epoch['epoch']) + 1],
        'mAP@50': [f"{final_epoch[map50_col]:.4f}"],
        'mAP@50-95': [f"{final_epoch[map_col]:.4f}"],
        'Precision': [f"{final_epoch[precision_col]:.4f}"],
        'Recall': [f"{final_epoch[recall_col]:.4f}"],
        'Parameters': ['3.2M'],
        'AMP': [TRAINING_CONFIG['amp']],
        'Status': ['✅ Baseline']
    })
    
    print("📊 PERFORMANCE SUMMARY")
    print("="*70)
    print(results_table.to_string(index=False))
    print("="*70 + "\n")
    
    # Save table
    results_table.to_csv(save_dir / "results_summary.csv", index=False)
    print(f"✅ Results table saved: {save_dir / 'results_summary.csv'}")
    
    # Performance comparison with EDA targets
    print(f"\n🎯 TARGET vs ACHIEVED:")
    print(f"="*70)
    
    target_map50 = 0.475  # Mid-point of 45-50% target
    achieved_map50 = final_epoch[map50_col]
    
    print(f"mAP@50:")
    print(f"   Target:   45-50% ({target_map50:.4f})")
    print(f"   Achieved: {achieved_map50*100:.2f}% ({achieved_map50:.4f})")
    
    if achieved_map50 >= 0.45:
        if achieved_map50 >= 0.50:
            print(f"   Status:   ✅ EXCEEDS TARGET!")
        else:
            print(f"   Status:   ✅ MEETS TARGET")
    else:
        print(f"   Status:   ⚠️  Below target (but still valid baseline)")
    
    print(f"="*70 + "\n")
    
    # Best epoch info
    best_map50_idx = df_results[map50_col].idxmax()
    best_epoch = df_results.iloc[best_map50_idx]
    
    print(f"📈 BEST EPOCH INFORMATION:")
    print(f"="*70)
    print(f"Best Epoch:     {int(best_epoch['epoch']) + 1}")
    print(f"Best mAP@50:    {best_epoch[map50_col]:.4f}")
    print(f"Best mAP@50-95: {best_epoch[map_col]:.4f}")
    print(f"Best Precision: {best_epoch[precision_col]:.4f}")
    print(f"Best Recall:    {best_epoch[recall_col]:.4f}")
    print(f"="*70 + "\n")

In [ ]:
"""
VISUALIZE TRAINING CURVES
"""

import matplotlib.pyplot as plt

print("📈 Generating training curves...\n")

# Load results
df_results = pd.read_csv(save_dir / "results.csv")

# Determine column names
map50_col = 'metrics/mAP50(B)' if 'metrics/mAP50(B)' in df_results.columns else 'metrics/mAP_0.5'
map_col = 'metrics/mAP50-95(B)' if 'metrics/mAP50-95(B)' in df_results.columns else 'metrics/mAP_0.5:0.95'
precision_col = 'metrics/precision(B)' if 'metrics/precision(B)' in df_results.columns else 'metrics/precision'
recall_col = 'metrics/recall(B)' if 'metrics/recall(B)' in df_results.columns else 'metrics/recall'

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle(f'Training Curves - {EXPERIMENT_NAME}', fontsize=18, fontweight='bold')

# 1. mAP curves
ax1 = axes[0, 0]
ax1.plot(df_results['epoch'], df_results[map50_col], label='mAP@50', linewidth=2, color='#2ecc71')
ax1.plot(df_results['epoch'], df_results[map_col], label='mAP@50-95', linewidth=2, color='#3498db')
ax1.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax1.set_ylabel('mAP', fontsize=12, fontweight='bold')
ax1.set_title('mAP Evolution', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(alpha=0.3)

# Mark best epoch
best_epoch_idx = df_results[map50_col].idxmax()
best_map50 = df_results.iloc[best_epoch_idx][map50_col]
best_epoch_num = df_results.iloc[best_epoch_idx]['epoch']
ax1.scatter([best_epoch_num], [best_map50], color='red', s=100, zorder=5, label='Best')
ax1.legend(fontsize=11)

# 2. Loss curves
ax2 = axes[0, 1]
loss_cols = [col for col in df_results.columns if 'loss' in col.lower() and 'train' in col.lower()]
colors = ['#e74c3c', '#f39c12', '#9b59b6']
for i, col in enumerate(loss_cols[:3]):
    label = col.replace('train/', '').replace('loss', '').strip()
    ax2.plot(df_results['epoch'], df_results[col], label=label, linewidth=2, color=colors[i % len(colors)])
ax2.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax2.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax2.set_title('Training Losses', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(alpha=0.3)

# 3. Precision & Recall
ax3 = axes[1, 0]
ax3.plot(df_results['epoch'], df_results[precision_col], label='Precision', linewidth=2, color='#3498db')
ax3.plot(df_results['epoch'], df_results[recall_col], label='Recall', linewidth=2, color='#e74c3c')
ax3.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax3.set_ylabel('Metric', fontsize=12, fontweight='bold')
ax3.set_title('Precision & Recall', fontsize=14, fontweight='bold')
ax3.legend(fontsize=11)
ax3.grid(alpha=0.3)

# 4. Learning rate
ax4 = axes[1, 1]
lr_cols = [col for col in df_results.columns if 'lr' in col.lower()]
for col in lr_cols:
    ax4.plot(df_results['epoch'], df_results[col], label=col.replace('lr/', 'LR: '), linewidth=2)
ax4.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax4.set_ylabel('Learning Rate', fontsize=12, fontweight='bold')
ax4.set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
if lr_cols:
    ax4.legend(fontsize=11)
ax4.grid(alpha=0.3)
ax4.set_yscale('log')

plt.tight_layout()
plt.savefig(save_dir / 'training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Training curves saved: {save_dir / 'training_curves.png'}\n")

# Update archive
print("📦 Updating archive with training curves...")
zip_path = saver.create_archive()

In [ ]:
"""
FINAL SUMMARY & NEXT STEPS
"""

print("╔" + "="*68 + "╗")
print("║" + " "*18 + "BASELINE TRAINING COMPLETE!" + " "*23 + "║")
print("╚" + "="*68 + "╝")

print(f"\n📊 Experiment: {EXPERIMENT_NAME}")
print(f"⏰ Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if 'final_metrics' in saver.metadata:
    metrics = saver.metadata['final_metrics']
    print(f"\n🎯 FINAL PERFORMANCE:")
    print(f"="*70)
    print(f"   mAP@50:      {metrics['mAP50']:.4f} ({metrics['mAP50']*100:.2f}%)")
    print(f"   mAP@50-95:   {metrics['mAP50-95']:.4f} ({metrics['mAP50-95']*100:.2f}%)")
    print(f"   Precision:   {metrics['precision']:.4f} ({metrics['precision']*100:.2f}%)")
    print(f"   Recall:      {metrics['recall']:.4f} ({metrics['recall']*100:.2f}%)")
    print(f"="*70)
    
    # Performance evaluation
    if metrics['mAP50'] >= 0.45:
        print(f"\n✅ SUCCESS: Baseline meets/exceeds target (45-50% mAP@50)")
    else:
        print(f"\n⚠️  Note: Below target but still valid baseline")

print(f"\n📁 ALL RESULTS SAVED TO:")
print(f"   Directory: {save_dir}")
print(f"   Archive:   {zip_path}")
print(f"   Size:      {Path(zip_path).stat().st_size / (1024*1024):.2f} MB")

print(f"\n🚨 CRITICAL: DOWNLOAD RESULTS NOW!")
print(f"="*70)
print(f"1. Go to left sidebar → Files")
print(f"2. Navigate to /kaggle/working/")
print(f"3. Find: {Path(zip_path).name}")
print(f"4. Right-click → Download")
print(f"5. Save to your computer")
print(f"="*70)

print(f"\n✅ What You've Accomplished:")
print(f"   ✅ Trained YOLOv8n baseline model")
print(f"   ✅ Achieved {metrics['mAP50']*100:.2f}% mAP@50")
print(f"   ✅ Generated 15+ training plots")
print(f"   ✅ Saved all weights and metrics")
print(f"   ✅ Established performance benchmark")
print(f"   ✅ Used AMP for optimal speed")

print(f"\n🚀 NEXT STEPS (Week 4):")
print(f"="*70)
print(f"1. 📥 Download and backup all results")
print(f"2. 📊 Document baseline in project log")
print(f"3. 🔬 Create new notebook: 'RDD2022 - YOLOv8s @ 640'")
print(f"4. 🎯 Target: 52-57% mAP@50 (7-12% improvement)")
print(f"5. ⏱️  GPU Time: ~15 hours")

print(f"\n📋 8-WEEK PROGRESS:")
print(f"="*70)
print(f"   ✅ Week 1:   EDA Complete")
print(f"   ✅ Week 2-3: Baseline (YOLOv8n @ 640) - {metrics['mAP50']*100:.2f}% mAP@50")
print(f"   ⏭️  Week 4:   Model Scaling (YOLOv8s @ 640)")
print(f"   ⏭️  Week 5-6: Resolution (YOLOv8s @ 1024)")
print(f"   ⏭️  Week 7:   Optimization (TTA + Tuning)")
print(f"   ⏭️  Week 8:   Documentation & Patent")

print(f"\n💡 Key Features Used:")
print(f"   • NumPy 1.26.4 (stable compatibility)")
print(f"   • AMP enabled (15% faster training)")
print(f"   • Auto-generated plots (15+ visualizations)")
print(f"   • Complete results archiving")
print(f"   • GPU: {torch.cuda.get_device_name(0)}")

print(f"\n🎓 Recommended Next Model:")
print(f"="*70)
print(f"   Model:      yolov8s.pt  (11.2M params vs 3.2M)")
print(f"   Resolution: 640×640     (same as baseline)")
print(f"   Epochs:     100         (same as baseline)")
print(f"   Batch:      16          (same as baseline)")
print(f"   Expected:   +7-12% mAP improvement")
print(f"="*70)

print("\n" + "="*70)
print("🎉 CONGRATULATIONS! Professional baseline complete!")
print("="*70 + "\n")

# Save this summary
summary_path = save_dir / "FINAL_SUMMARY.txt"
with open(summary_path, 'w') as f:
    f.write(f"""
BASELINE TRAINING SUMMARY
{'='*70}

Experiment:     {EXPERIMENT_NAME}
Completed:      {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

PERFORMANCE:
   mAP@50:      {metrics['mAP50']:.4f} ({metrics['mAP50']*100:.2f}%)
   mAP@50-95:   {metrics['mAP50-95']:.4f} ({metrics['mAP50-95']*100:.2f}%)
   Precision:   {metrics['precision']:.4f} ({metrics['precision']*100:.2f}%)
   Recall:      {metrics['recall']:.4f} ({metrics['recall']*100:.2f}%)

CONFIGURATION:
   Model:       YOLOv8n
   Resolution:  640×640
   Epochs:      {TRAINING_CONFIG['epochs']}
   Batch:       {TRAINING_CONFIG['batch']}
   Parameters:  3.2M
   AMP:         {TRAINING_CONFIG['amp']}
   Plots:       {TRAINING_CONFIG['plots']}

STATUS: ✅ BASELINE ESTABLISHED

NEXT: Scale to YOLOv8s for improved performance
""")

print(f"✅ Final summary saved: {summary_path}")

# Final archive update
print(f"\n📦 Creating final archive...")
zip_path = saver.create_archive()

print(f"\n{'='*70}")
print(f"✅ ALL DONE! Download {Path(zip_path).name} NOW!")
print(f"{'='*70}\n")